In [7]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import base64

forest_path = "forest_change/annual-change-forest-area.csv"
df = pd.read_csv(forest_path)

df.columns = [c.strip() for c in df.columns]
df = df.rename(columns={
    "Annual net change in forest area": "change",
    "Entity": "country",
    "Code": "iso_code",
    "Year": "year"
})
df["year"] = pd.to_numeric(df["year"], errors="coerce").astype(int)
df = df[(df["year"] >= 1990) & (df["year"] <= 2015)]
years = sorted(df["year"].unique())

owid_url = "https://github.com/owid/co2-data/raw/master/owid-co2-data.csv"
owid = pd.read_csv(owid_url,
                   usecols=["iso_code", "year", "co2", "land_use_change_co2", "temperature_change_from_ghg"],
                   low_memory=False)
owid = owid[owid["iso_code"].str.len() == 3]
owid["year"] = owid["year"].astype(int)

df = df.merge(owid, how="left", on=["iso_code", "year"])
df["co2_value"] = df["co2"].fillna(0.0)
df["co2_fossil"] = df["co2_value"]
df["co2_luc"] = df["land_use_change_co2"].fillna(0.0)
df["temp_change"] = df["temperature_change_from_ghg"]

if df["change"].dropna().shape[0] > 0:
    p5, p95 = np.percentile(df["change"].dropna(), [5, 95])
else:
    p5, p95 = -1, 1
mx = max(abs(p5), abs(p95), 1)
vmin, vmax = -mx, mx

SAMPLE_YEARS = [1990, 2000, 2010, 2015]

fig = make_subplots(
    rows=3, cols=2,
    column_widths=[0.34, 0.66],
    row_heights=[0.33, 0.33, 0.33],
    specs=[
        [{"type": "xy"}, {"type": "choropleth", "rowspan": 2}],
        [{"type": "xy"}, None],
        [{"type": "xy"}, {"type": "xy"}]
    ],
    horizontal_spacing=0.04,
    vertical_spacing=0.08
)

def temp_series(country_name):
    g = df[df["country"] == country_name].sort_values("year")
    return g["year"].tolist(), g["temp_change"].tolist()

def co2_bar_series(country_name):
    g = df[df["country"] == country_name]
    g = g[g["year"].isin(SAMPLE_YEARS)].sort_values("year")
    return g["year"].tolist(), g["co2_fossil"].abs().tolist(), g["co2_luc"].abs().tolist()

country_list = sorted(df["country"].dropna().unique())
default_country = "Brazil" if "Brazil" in country_list else (country_list[0] if country_list else None)
if default_country is None:
    raise ValueError("No countries found in the dataset.")

gmsl_csv = "Global_sea_level_rise.csv"
g_all = pd.read_csv(gmsl_csv)
cols = {c.lower().strip(): c for c in g_all.columns}
year_col = None; mm_col = None
for cand in ("year","yr","# year","date","calendar_year"):
    if cand in cols:
        year_col = cols[cand]; break
for cand in ("mmfrom1993-2022","mmfrom1993-2008average","sea level difference in mm","mm","value","sea_level_mm","mm_from_baseline"):
    if cand in cols:
        mm_col = cols[cand]; break
if year_col is None:
    for c in g_all.columns:
        if np.issubdtype(g_all[c].dtype, np.integer) and g_all[c].min()>1800 and g_all[c].max()<3000:
            year_col = c; break
if mm_col is None:
    for c in g_all.columns:
        if c!=year_col and np.issubdtype(g_all[c].dtype, np.number):
            mm_col = c; break
if year_col is None or mm_col is None:
    raise ValueError("Global_sea_level_rise.csv: cannot detect year/mm columns.")

g = g_all.rename(columns={year_col:"year", mm_col:"gmsl_mm"})
g["year"] = pd.to_numeric(g["year"], errors="coerce").astype(int)
g = g.sort_values("year").reset_index(drop=True)
g_sel = g[(g["year"]>=1990)&(g["year"]<=2015)].copy()
if g_sel.empty:
    raise ValueError("Global_sea_level_rise.csv: no rows 1990..2015 found.")

x_years = g_sel["year"].tolist()
y_series = g_sel["gmsl_mm"].values
val_1990 = float(g_sel[g_sel["year"]==1990]["gmsl_mm"].iloc[0]) if 1990 in g_sel["year"].values else float(np.interp(1990, g_sel["year"], g_sel["gmsl_mm"]))
val_2005 = float(g_sel[g_sel["year"]==2005]["gmsl_mm"].iloc[0]) if 2005 in g_sel["year"].values else float(np.interp(2005, g_sel["year"], g_sel["gmsl_mm"]))
y_rel = (y_series - val_1990).tolist()
y_mid = (np.array([val_2005 - val_1990] * len(x_years))).tolist()
y_base = [0.0]*len(x_years)


SCALE_TEMP = 1_000_000.0  # 1e6 -> µ°C

x_temp_def, t_def = temp_series(default_country)
def _scale_list(lst, scale):
    out = []
    for v in lst:
        if v is None or (isinstance(v, float) and np.isnan(v)):
            out.append(None)
        else:
            out.append(float(v) * scale)
    return out

t_def_scaled = _scale_list(t_def, SCALE_TEMP)

fig.add_trace(go.Scatter(x=x_temp_def, y=t_def_scaled, mode="lines+markers",
                         name="Temperature change (µ°C)", showlegend=False),
              row=1, col=1)
fig.update_xaxes(title_text="Year", row=1, col=1,   title_font=dict(size=12), title_standoff=0)

fig.update_yaxes(
    title_text="Temperature change (µ°C)",
    row=1, col=1,
    autorange=True,
    tickformat=".0f",
    title_font=dict(size=10)
)

x_bar_def, fossil_def, luc_def = co2_bar_series(default_country)
fig.add_trace(go.Bar(x=x_bar_def, y=fossil_def, name="Fossil CO₂ (Mt)"), row=2, col=1)
fig.add_trace(go.Bar(x=x_bar_def, y=luc_def, name="Land-use CO₂ (Mt)"), row=2, col=1)
fig.update_xaxes(title_text="Year", type="category", categoryorder="array",
                 title_standoff=0, categoryarray=SAMPLE_YEARS, row=2, col=1, title_font=dict(size=12))
fig.update_yaxes(title_text="CO₂ emissions (million tonnes)", row=2, col=1, title_font=dict(size=10))
fig.update_layout(barmode="group")

fig.add_trace(go.Scatter(x=x_years, y=y_base, mode="lines", line=dict(width=0),
                         fill="tozeroy", fillcolor="rgba(20,20,40,0.04)",
                         name="Sea level baseline", hoverinfo="skip", showlegend=False),
              row=3, col=1)
fig.add_trace(go.Scatter(x=x_years, y=y_mid, mode="lines",
                         line=dict(width=1.0, dash="dot", color="rgb(40,40,60)", shape="spline"),
                         fill="tonexty", fillcolor="rgba(40,40,60,0.12)",
                         name="Sea level (mid)", hoverinfo="skip", showlegend=True),
              row=3, col=1)
fig.add_trace(go.Scatter(x=x_years, y=y_rel, mode="lines",
                         line=dict(width=2.5, color="rgb(30,30,50)", shape="spline"),
                         fill="tonexty", fillcolor="rgba(30,30,50,0.18)",
                         name="Sea level (1990→2015)",
                         hovertemplate="Year %{x}<br>Δ from 1990: %{y:.2f} mm<extra></extra>",
                         showlegend=True),
              row=3, col=1)
ymin = 0.0
ymax = float(np.nanmax(y_rel)) if len(y_rel) > 0 else 0.0
if ymax > 0:
    stripe_step = max(2.0, (ymax - ymin) / 18.0)
    stripes = []
    for yv in np.arange(ymin + stripe_step*0.5, ymax, stripe_step):
        stripes.append(dict(type="line", xref="x", yref="y",
                            x0=min(x_years), x1=max(x_years),
                            y0=float(yv), y1=float(yv),
                            line=dict(color="rgba(20,20,40,0.06)", width=1),
                            layer="above"))
    existing_shapes = list(fig.layout.shapes) if ("shapes" in fig.layout and fig.layout.shapes is not None) else []
    fig.update_layout(shapes=existing_shapes + stripes)

fig.update_xaxes(title_text="Year", row=3, col=1, title_font=dict(size=12),
                 tickmode="array", tickvals=[1990,1995,2000,2005,2010,2015])
fig.update_yaxes(title_text="Sea level change since 1990 (mm)", row=3, col=1, title_font=dict(size=10))

y0 = years[0]
df0 = df[df["year"] == y0].copy()
df0['co2_value_safe'] = df0.get('co2_value', 0.0).fillna(0.0)
custom0 = df0[["iso_code", "country", "co2_value_safe"]].apply(lambda r: [r["iso_code"], r["country"], float(r["co2_value_safe"])], axis=1).tolist()

map_trace = go.Choropleth(
    locations=df0["iso_code"],
    z=df0["change"],
    customdata=custom0,
    text=df0.apply(lambda r: f"{r['country']}<br>{r['year']}: {r['change']:,.0f} ha"
                         f"<br>CO₂: {r['co2_value_safe']:,} Mt", axis=1),
    locationmode="ISO-3",
    zmin=vmin, zmax=vmax, zmid=0,
    colorscale="RdYlGn",
    marker_line_color="white",
    marker_line_width=0.2,
    colorbar=dict(title="Net forest change (ha)", x=0.92, thickness=16)
)
fig.add_trace(map_trace, row=1, col=2)
fig.layout.geo.domain = dict(x=[0.30, 0.98], y=[0.05, 0.95])

fig.update_geos(projection_type="orthographic", projection_scale=1.0,
                showcountries=True, showcoastlines=True, showland=True,
                landcolor="rgb(230,230,230)", showocean=True, oceancolor="rgb(180,200,250)",
                row=1, col=2)
fig.update_layout(dragmode="pan")

frames = []
map_trace_index = None
for i, t in enumerate(fig.data):
    if getattr(t, "type", "") == "choropleth":
        map_trace_index = i
        break

for y in years:
    dfy = df[df["year"] == y].copy()
    dfy['co2_value_safe'] = dfy.get('co2_value', 0.0).fillna(0.0)
    customy = dfy[["iso_code", "country", "co2_value_safe"]].apply(lambda r: [r["iso_code"], r["country"], float(r["co2_value_safe"])], axis=1).tolist()
    chor = go.Choropleth(
        locations=dfy["iso_code"],
        z=dfy["change"],
        customdata=customy,
        text=dfy.apply(lambda r: f"{r['country']}<br>{r['year']}: {r['change']:,.0f} ha"
                             f"<br>CO₂: {r['co2_value_safe']:,} Mt", axis=1),
        locationmode="ISO-3", zmin=vmin, zmax=vmax, zmid=0,
        colorscale="RdYlGn", marker_line_color="white", marker_line_width=0.2,
        colorbar=dict(title="Net forest change (ha)", x=0.86, thickness=18)
    )
    frames.append(go.Frame(name=str(y), data=[chor], traces=[map_trace_index] if map_trace_index is not None else None))
fig.frames = frames

left_trace_indices = [0, 1, 2] 

dropdown_buttons = []
for c in country_list:
    xt, yt = temp_series(c)
    xb, yf, yl = co2_bar_series(c)
    yt_scaled = _scale_list(yt, SCALE_TEMP)

    x_updates = [xt, xb, xb]
    y_updates = [yt_scaled, yf, yl]
    dropdown_buttons.append(dict(label=c, method="restyle", args=[{"x": x_updates, "y": y_updates}, left_trace_indices]))

fig.update_layout(
    updatemenus=[
        dict(type="buttons", showactive=False, x=0.40, y=0.08, xanchor="center", yanchor="middle",
             buttons=[dict(label="Play", method="animate",
                           args=[None, {"frame": {"duration": 800, "redraw": True},
                                        "transition": {"duration": 300}, "fromcurrent": True}]),
                      dict(label="Pause", method="animate", args=[[None], {"frame": {"duration": 0, "redraw": False}}])
                      ]),
        dict(type="dropdown", showactive=True, x=0.009, y=1.00, xanchor="left", yanchor="top",
             buttons=dropdown_buttons, font=dict(size=10))
    ],
    sliders=[dict(active=0, pad={"t":20}, steps=[dict(method="animate",
                                                     args=[[str(y)], {"frame": {"duration": 0, "redraw": True}, "mode": "immediate"}],
                                                     label=str(y)) for y in years], x=0.35, len=0.55)],
    title=dict(text="Deforestation and Its Climate Footprint (1990–2015)", x=0.50, y=0.98, xanchor="center"),
    margin=dict(l=60, r=30, t=100, b=60),
    height=680, width=1500
)

fig.update_layout(showlegend=True, legend=dict(orientation="h", x=0.00, y=1.08, xanchor="left", yanchor="top",
                                               bgcolor="rgba(255,255,255,0.6)", font=dict(size=13),
                                               bordercolor="rgba(0,0,0,0.1)", borderwidth=1))
exhaust_svg = """
<svg xmlns='http://www.w3.org/2000/svg' width='220' height='220' viewBox='0 0 220 220'>
 <defs>
  <linearGradient id='g1' x1='0' x2='1' y1='0' y2='1'>
    <stop offset='0' stop-color='#666'/>
    <stop offset='1' stop-color='#222'/>
  </linearGradient>
  <radialGradient id='s1' cx='50%' cy='20%' r='60%'>
    <stop offset='0' stop-color='#ddd' stop-opacity='0.9'/>
    <stop offset='1' stop-color='#aaa' stop-opacity='0.2'/>
  </radialGradient>
 </defs>
 <rect x='80' y='100' width='60' height='80' rx='8' fill='url(#g1)'/>
 <rect x='70' y='110' width='80' height='20' rx='10' fill='#111'/>
 <ellipse cx='110' cy='95' rx='40' ry='12' fill='#333'/>
 <g fill='url(#s1)' opacity='0.95'>
   <circle cx='110' cy='70' r='18'/>
   <circle cx='128' cy='54' r='14'/>
   <circle cx='92' cy='52' r='12'/>
   <circle cx='140' cy='36' r='10'/>
 </g>
</svg>
"""
svg_bytes = exhaust_svg.encode("utf-8")
svg_b64 = base64.b64encode(svg_bytes).decode("ascii")
svg_data_url = "data:image/svg+xml;base64," + svg_b64

injected_js = f"""
<script>
(function() {{
  const svgUrl = "{svg_data_url}";
  function ensureOverlay() {{
    var el = document.getElementById("exhaust_overlay_img");
    if (!el) {{
      el = document.createElement("img");
      el.id = "exhaust_overlay_img";
      el.style.position = "absolute";
      el.style.pointerEvents = "none";
      el.style.transition = "transform 600ms ease, opacity 600ms ease";
      el.style.opacity = "0";
      el.style.transformOrigin = "50% 100%";
      el.style.zIndex = 1000;
      document.body.appendChild(el);
    }}
    return el;
  }}
  function showExhaustAt(x, y, scale) {{
    var el = ensureOverlay();
    el.src = svgUrl;
    var size = Math.max(50, Math.min(320, Math.round(80 * scale)));
    el.style.width = size + "px";
    el.style.height = "auto";
    el.style.left = (x - size/2) + "px";
    el.style.top = (y - size + 20) + "px";
    requestAnimationFrame(function() {{ el.style.opacity = "1"; el.style.transform = "scale(1.0)"; }});
    clearTimeout(window.___exhaustTimeout);
    window.___exhaustTimeout = setTimeout(function() {{ el.style.opacity = "0"; }}, 3000);
  }}
  function attachClick() {{
    var plotDiv = document.querySelector(".plotly-graph-div");
    if (!plotDiv) {{ setTimeout(attachClick, 200); return; }}
    var minCo = Infinity, maxCo = -Infinity;
    try {{
      var data = plotDiv.data || [];
      for (var t = 0; t < data.length; t++) {{
        var cd = data[t].customdata;
        if (!cd) continue;
        for (var i = 0; i < cd.length; i++) {{
          var v = parseFloat(cd[i][2]);
          if (!isNaN(v)) {{ if (v < minCo) minCo = v; if (v > maxCo) maxCo = v; }}
        }}
      }}
    }} catch(e) {{}}
    if (!isFinite(minCo)) minCo = 0; if (!isFinite(maxCo)) maxCo = minCo === 0 ? 1 : maxCo;
    plotDiv.on("plotly_click", function(ev) {{
      if (!ev || !ev.points || !ev.points.length) return;
      var pt = ev.points[0];
      var cd = pt.customdata || (pt.data && pt.data.customdata && pt.data.customdata[pt.pointIndex]);
      if (!cd) return;
      var co2 = parseFloat(cd[2]) || 0;
      var norm = maxCo > minCo ? (co2 - minCo) / (maxCo - minCo) : 0;
      norm = Math.max(0, Math.min(1, norm));
      var scale = 0.6 + norm*0.7;
      var e = ev.event || window.event;
      var x = e && e.clientX ? e.clientX : window.innerWidth/2;
      var y = e && e.clientY ? e.clientY : window.innerHeight/2;
      showExhaustAt(x, y, scale);
    }});
  }}
  window.addEventListener("load", function() {{ setTimeout(attachClick, 200); }});
}})();
</script>
"""

full_html = fig.to_html(full_html=True, include_plotlyjs="cdn")
full_html = full_html.replace("</body>", injected_js + "</body>")
output_file = "Assignment3.html"
with open(output_file, "w", encoding="utf-8") as f:
    f.write(full_html)

print("Wrote:", output_file)
print("Open this file in a browser (Chrome/Edge/Firefox).")


Wrote: Assignment3.html
Open this file in a browser (Chrome/Edge/Firefox).
